# Azure Cloud

Панель управления облаком через браузер - https://portal.azure.com/#home

### Создаем виртуальную машину

"Создать ресурс" -> "Ubuntu Server 18.04 LTS"

* **ВАЖНО** - укажите правильную подписку, созданную специально для этого курса!
* Группа ресурсов - это специальная группа, к которой будут подключены все штуки, которые вы будете создавать в облаке. Для работы вам будет достаточно пользоваться одной группой ресурсов для всего. Создайте новую (или если уже есть, выберете существующую)
* Подробности об экземпляре - можно выдать любое имя машины и оставить остальные параметры как есть
* Учетная запись администратора - особой разницы между ключем SSH и паролем нет - выбирайте любой, какой больше нравится (не забудьте пароль, если выбрали его)
* Входящие порты - оставьте только 22. Далее мы еще узнаем, как получить безопасный доступ к машине

-- Далее --

* Диск лучше оставить SSD
* Если на машине будете работать с большими объемами, то можно подключить новый диск

-- Далее -- 

* Сетевые подключения - оставляем как есть

-- Далее -- 

* Управление - оставлякм как есть

-- Далее -- 

* Дополнительно - оставляем как есть

-- Далее -- 

* Теги - оставляем как есть

-- Далее --

* СОЗДАТЬ!

Если у вас создавался новый SSH ключ, то скачайте его и положите в `~/.ssh`, после чего
```bash
sudo chmod 600 ~/.ssh/KEY.pem
eval $(ssh-agent -s)
ssh-add ~/.ssh/KEY.pem
```

После того, как машина развернется в облаке, можно к ней подключится. На странице ресурса необходимо получить "Общедоступный IP-адрес".

```bash
ssh azureuser@1.2.3.4
```

Чтобы получить права администратора, достаточно выполнить

```bash
sudo su
```

Поздравляю, это ваша личная виртуальная машина в облаке!

### Подключаем диски

Если вы добавляли диск на машину, то он автоматически не примонтируется к операционной системе. Для того, чтобы им воспользоваться нужно

Проверить, что блочное устройство подключено:

```bash
$ sudo fdisk -l

...
Disk /dev/sda: 1 TiB, 1099511627776 bytes, 2147483648 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 4096 bytes
I/O size (minimum/optimal): 4096 bytes / 4096 bytes
```

Выделить раздел на устройстве

```bash
$ sudo parted /dev/sda --script mklabel gpt mkpart primary ext4 0% 100%
$ sudo mkfs -t ext4 /dev/sda1
$ sudo fdisk -l

...
Device     Start        End    Sectors  Size Type
/dev/sda1   2048 2147481599 2147479552 1024G Linux filesystem
```

Примонтировать

```bash
$ sudo mkdir /data
$ sudo mount /dev/sda1 /data
```

Ваш диск теперь должен быть в `/data`

### Ставим инструменты для работы

Если вы планирете заниматься на машине анализом данных, то скорее всего это будет удобнее делать через Jupyter Notebook. 

Ставим библиотеку

```bash
# get link from this site - https://www.anaconda.com/distribution/

wget https://repo.anaconda.com/archive/Anaconda3-2020.11-Linux-x86_64.sh -O install-conda.sh
chmod +x install-conda.sh
./install-conda.sh

# add conda to PATH

eval "$(/home/azureuser/anaconda3/bin/conda shell.bash hook)"
conda init
```

Запускаем Jupyter. Чтобы он продолжил работать даже после того, как вы закроете терминал, используйте `tmux`. Подсказки по использованию tmux - https://gist.github.com/MohamedAlaa/2961058 .

```bash
tmux new -s jupyter
jupyter notebook --ip=0.0.0.0 --NotebookApp.token=
```

### Подключаемся через Proxy

В целях безопасности, никакого доступа к машине, кроме как по SSH, нет. Чтобы получить доступ до ноутбука (или до любых других сетевых компонентов на машине) нужно подключить Proxy через SSH.

Для этого поднимаем SOCKS proxy

```bash
ssh -4 -D <local-port> <user>@<remotehost> -N
```

* -D <local-port> указывает на каком порту на **локальной машине** открыть прокси
* -N - не открывать shell на удаленной машине - только прокси
* -4 - использовать IPv4
    
После устанавливаем расширение для Chrome Proxy SwitchyOmega - https://chrome.google.com/webstore/detail/proxy-switchyomega/padekgcemlokbadohgkifijomclgjgif?hl=en-GB
    
Внутри Omega нужно настроить

* Proxy - SOCKS5 | localhost | 5555 - ходить в **локалхост** за проксей
* Auto switch - Host wildcard | 10.0.0.* | proxy - на всех адресах совпадающих с паттерном использовать прокси
    
Note: 10.0.0.* - это адреса внутренней сети ваших машин в облаке. Это можно проверить в панели управления.
    
Пример моей настройки прокси

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/proxy-1.png">
<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/proxy-2.png">
    


### Примерная схема того, как работает доступ до облака через прокси

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/proxy-azure.png">

# AZ CLI

Каждый раз руками тыкать на все кнопки в интерфейсе - это неудобно. Для того, чтобы работать с облаком в более автоматическом режиме, Azure предоставляет утилиту командной строки `az`.

Установим ее и посмотрим, как она работает.

In [14]:
! brew update && brew install azure-cli

Already up-to-date.
==> Downloading https://homebrew.bintray.com/bottles/openssl%401.1-1.1.1i.mojave
Already downloaded: /Users/arinaruck/Library/Caches/Homebrew/downloads/e292d9b7a7ac0068d280a7c5df4e58f408f96d02e65cbf187a24d640db76e971--openssl@1.1-1.1.1i.mojave.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/gdbm-1.18.1_1.mojave.bottle
Already downloaded: /Users/arinaruck/Library/Caches/Homebrew/downloads/96af53ea7fe78d119b6b764d60e7f4b1017ba58c396fc17fa9fd99d75b68272e--gdbm-1.18.1_1.mojave.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/readline-8.1.mojave.bottle.
Already downloaded: /Users/arinaruck/Library/Caches/Homebrew/downloads/7a26dc754eeb33d34e127accbf2a444d3c601f8e08b8033416a3bd497a863997--readline-8.1.mojave.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/sqlite-3.34.0.mojave.bottle
Already downloaded: /Users/arinaruck/Library/Caches/Homebrew/downloads/177ac8a45e671efb85bd1565ad1a6ebf18d2fb81f38743d1832f90aa52459d78-

Pruned 0 symbolic links and 2 directories from /usr/local
==> Upgrading 2 dependents:
pyenv 1.2.20 -> 1.2.22, wget 1.20.3_2 -> 1.21
==> Upgrading pyenv 1.2.20 -> 1.2.22 
==> Downloading https://homebrew.bintray.com/bottles/pyenv-1.2.22.mojave.bottle.
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/525b4cc06fb46fd1e86d2
######################################################################## 100.0%
==> Pouring pyenv-1.2.22.mojave.bottle.tar.gz
🍺  /usr/local/Cellar/pyenv/1.2.22: 721 files, 2.5MB
Removing: /usr/local/Cellar/pyenv/1.2.20... (708 files, 2.5MB)
==> Upgrading wget 1.20.3_2 -> 1.21 
==> Downloading https://homebrew.bintray.com/bottles/gettext-0.21.mojave.bottle.
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/99707d4dcc731faf98033
######################################################################## 100.0%
==> Downloading https://homebrew.bintray.com/bottles/wget-1.21.mojave.bottle.tar
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/6343b9c7

In [15]:
%%bash

az --version

azure-cli                         2.18.0

core                              2.18.0
telemetry                          1.0.6

Python location '/usr/local/Cellar/azure-cli/2.18.0/libexec/bin/python'
Extensions directory '/Users/arinaruck/.azure/cliextensions'

Python (Darwin) 3.8.7 (default, Dec 30 2020, 10:13:09) 
[Clang 11.0.0 (clang-1100.0.33.17)]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/azureclihats
and let us know if you're interested in trying out our newest features: https://aka.ms/CLIUXstudy


In [16]:
! az login

The default web browser has been opened at https://login.microsoftonline.com/common/oauth2/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "id": "dfeaef04-2e5c-4480-8ebf-301863ed526a",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Microsoft Azure Sponsorship 2",
    "state": "Enabled",
    "tenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "user": {
      "name": "arina.ruck@gmail.com",
      "type": "user"
    }
  }
]


In [17]:
%%writefile password-file.txt
Aa123456

Writing password-file.txt


In [18]:
! cat password-file.txt | head -c 1

A

In [19]:
! az login -u arina.ruck@gmail.com -p $(cat password-file.txt)

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "id": "dfeaef04-2e5c-4480-8ebf-301863ed526a",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Microsoft Azure Sponsorship 2",
    "state": "Enabled",
    "tenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "user": {
      "name": "arina.ruck@gmail.com",
      "type": "user"
    }
  }
]


In [22]:
! az account list

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "id": "dfeaef04-2e5c-4480-8ebf-301863ed526a",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Microsoft Azure Sponsorship 2",
    "state": "Enabled",
    "tenantId": "fa34e92c-e581-439a-b6ae-b41b61270513",
    "user": {
      "name": "arina.ruck@gmail.com",
      "type": "user"
    }
  }
]


С помощью этого инструмента из командной строки можно проводить любые манипуляции над ресурсами в облаке. Однако почти всегда элементы в облаке требуют большого числа параметров и задавать их в одной команде очень неудобно. Например так выглядит документация по команде `az vm create`, которая создает новую виртуальную машину - [смотри в доке](https://docs.microsoft.com/en-us/cli/azure/vm?view=azure-cli-latest#az_vm_create).

```bash
az vm create --name
             --resource-group
             [--accelerated-networking {false, true}]
             [--admin-password]
             [--admin-username]
             [--asgs]
             [--assign-identity]
             [--attach-data-disks]
             [--attach-os-disk]
             [--authentication-type {all, password, ssh}]
             [--availability-set]
             [--boot-diagnostics-storage]
             [--computer-name]
             [--custom-data]
             [--data-disk-caching]
             [--data-disk-encryption-sets]
             [--data-disk-sizes-gb]
             [--enable-agent {false, true}]
             [--enable-auto-update {false, true}]
             [--encryption-at-host {false, true}]
             [--ephemeral-os-disk {false, true}]
             [--eviction-policy {Deallocate, Delete}]
             [--generate-ssh-keys]
             [--host]
             [--host-group]
             [--image]
             [--license-type {None, RHEL_BYOS, SLES_BYOS, Windows_Client, Windows_Server}]
             [--location]
             [--max-price]
             [--nics]
             [--no-wait]
             [--nsg]
             [--nsg-rule {NONE, RDP, SSH}]
             [--os-disk-caching {None, ReadOnly, ReadWrite}]
             [--os-disk-encryption-set]
             [--os-disk-name]
             [--os-disk-size-gb]
             [--os-type {linux, windows}]
             [--patch-mode {AutomaticByOS, AutomaticByPlatform, Manual}]
             [--plan-name]
             [--plan-product]
             [--plan-promotion-code]
             [--plan-publisher]
             [--ppg]
             [--priority {Low, Regular, Spot}]
             [--private-ip-address]
             [--public-ip-address]
             [--public-ip-address-allocation {dynamic, static}]
             [--public-ip-address-dns-name]
             [--public-ip-sku {Basic, Standard}]
             [--role]
             [--scope]
             [--secrets]
             [--size]
             [--specialized {false, true}]
             [--ssh-dest-key-path]
             [--ssh-key-values]
             [--storage-account]
             [--storage-container-name]
             [--storage-sku]
             [--subnet]
             [--subnet-address-prefix]
             [--subscription]
             [--tags]
             [--ultra-ssd-enabled {false, true}]
             [--use-unmanaged-disk]
             [--validate]
             [--vmss]
             [--vnet-address-prefix]
             [--vnet-name]
             [--workspace]
             [--zone {1, 2, 3}]
```

Поэтому мы с вами пойдем немного более продвинутым путем.

# Terraform

Для удобной автоматизации задач по развертке ресурсов в облаке существуют различные Infrastructure-as-a-Code решения. В рамках этой парадигмы пользователю необходимо написать определенную конфигурацию облачных ресурсов, а специальный инструмент за пользователя воссоздаст необходимые элементы.

Для Azure существует как минимум два популярных решения такого рода - Шаблоны Azure Resource Manager (ARM) и Terraform (подробнее можно почитать [в документации Azure](https://docs.microsoft.com/ru-ru/dotnet/architecture/cloud-native/infrastructure-as-code) ). Мы с вами будем использовать Terraform, как более удобный и универсальный инструмент автоматизации.

Более детально про язык конфигурации HCL, используемый в Terraform, и про сам инструмент можно прочитать на [официальном сайте разработчиков](https://www.terraform.io/). Мы рассмотрим основные возможности.

### Начало работы

In [65]:
# Скачиваем инструмент и кладем его в /usr/bin.

! brew tap hashicorp/tap
! brew install hashicorp/tap/terraform

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> Updated Formulae
Updated 1 formula.

==> Tapping hashicorp/tap
Cloning into '/usr/local/Homebrew/Library/Taps/hashicorp/homebrew-tap'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 763 (delta 88), reused 68 (delta 35), pack-reused 604
Receiving objects: 100% (763/763), 151.67 KiB | 976.00 KiB/s, done.
Resolving deltas: 100% (351/351), done.
Tapped 8 formulae (43 files, 234.6KB).
Updating Homebrew...
==> Installing terraform from hashicorp/tap
==> Downloading https://releases.hashicorp.com/terraform/0.14.4/terraform_0.14.4
######################################################################## 100.0%                                                                          2.1%                                                            4.3%
🍺  /usr/local/Cellar/terraform/0.14.4: 3 files, 78.4MB, bu

In [67]:
! terraform -help

Usage: terraform [global options] <subcommand> [args]

The available commands for execution are listed below.
The primary workflow commands are given first, followed by
less common or more advanced commands.

Main commands:
  init          Prepare your working directory for other commands
  validate      Check whether the configuration is valid
  plan          Show changes required by the current configuration
  apply         Create or update infrastructure
  destroy       Destroy previously-created infrastructure

All other commands:
  console       Try Terraform expressions at an interactive command prompt
  fmt           Reformat your configuration in the standard style
  force-unlock  Release a stuck lock on the current workspace
  get           Install or upgrade remote Terraform modules
  graph         Generate a Graphviz graph of the steps in an operation
  import        Associate existing infrastructure with a Terraform resource
  login         Obtain and save credentials for a

In [68]:
# Работать будем из новой директории

! mkdir -p my-first-cloud

%cd my-first-cloud/

/Users/arinaruck/Desktop/cs/lsml/lsml-2021-public/my-first-cloud/my-first-cloud


In [69]:
! pwd
! ls

/Users/arinaruck/Desktop/cs/lsml/lsml-2021-public/my-first-cloud/my-first-cloud


In [70]:
! terraform init

Terraform initialized in an empty directory!

The directory has no Terraform configuration files. You may begin working
with Terraform immediately by creating Terraform configuration files.


В файлах с расширением `tf` описываются ресурсы, которые необходимо создать в облаке. Какие ресурсы вообще доступны и какие параметры они требуют можно посмотреть в [официальной документации Terraform для Azure](https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs).

In [13]:
%%writefile common.tf

provider "azurerm" {
  # Провайдер, с которым будем работать. В данном случае - Azure
  version = "=2.40.0"
  features {}
}

# Группа ресурсов. К ней будут линковаться все осталные ресурсы
resource "azurerm_resource_group" "lsml_rg" {
  name = "lsml-resource-group"
  location = "westus"
}

# Виртуальная сеть внутри облака
resource "azurerm_virtual_network" "lsml_vn" {
  name = "lsml-vitrual-network"
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location
  address_space = ["10.0.0.0/16"]   # Пул адресов внутри сети
}

Writing common.tf


In [14]:
! terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "2.40.0"...
- Installing hashicorp/azurerm v2.40.0...
- Installed hashicorp/azurerm v2.40.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.



  on common.tf line 4, in provider "azurerm":
   4:   version = "=2.40.0"

Terraform 0.13 and earlier allowed provider version constraints inside the
provider configuration block, but that is now deprecated and will be removed
in a future version of Terraform. To silence this warning, move the provider
version constraint into the required_providers block.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that a

In [15]:
! ls -la

total 20
drwxr-xr-x  3 root root 4096 Jan 13 12:26 .
drwxrwxr-x 11 1001 1002 4096 Jan 13 12:25 ..
-rw-r--r--  1 root root  723 Jan 13 12:26 common.tf
drwxr-xr-x  3 root root 4096 Jan 13 12:26 .terraform
-rwxrwxrwx  1 root root 1035 Jan 13 12:26 .terraform.lock.hcl


Удалим все ресурсы в облаке через интерфейс. После этого запускаем команду 
```bash
terraform apply
```

Она посмотрит на конфигурацию, посмотрим на текущее состояние дел в облаке и составит план работ. 
Если план вас устраивает, то необходимо в явном виде согласиться с применением этого плана. По этой причине лучше запускать этот инструмент из командной строки, а не из Jupyter Notebook. Мы будем использовать трюк пайплайнами в Bash. Про то, почему это работает и как этим пользоваться - в следующий раз.

Note: если Terraform жалуется на авторизацию, запустите `az login` перед terraform.

In [16]:
! terraform plan  # Посмотреть только план, без реальных изменений


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.lsml_rg will be created
  + resource "azurerm_resource_group" "lsml_rg" {
      + id       = (known after apply)
      + location = "westus"
      + name     = "lsml-resource-group"
    }

  # azurerm_virtual_network.lsml_vn will be created
  + resource "azurerm_virtual_network" "lsml_vn" {
      + address_space         = [
          + "10.0.0.0/16",
        ]
      + guid                  = (known after apply)
      + id                    = (known after apply)
      + location              = "westus"
      + name                  = "lsml-vitrual-network"
      + resource_group_name   = "lsml-resource-group"
      + subnet                = (known after apply)
      + vm_protection_enabled = false
    }

Plan: 2 to add, 0 to change, 0 to destroy.


  on common.tf line 4, in provider "az

In [17]:
! echo "yes" | terraform apply  # Строим и применяем план


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.lsml_rg will be created
  + resource "azurerm_resource_group" "lsml_rg" {
      + id       = (known after apply)
      + location = "westus"
      + name     = "lsml-resource-group"
    }

  # azurerm_virtual_network.lsml_vn will be created
  + resource "azurerm_virtual_network" "lsml_vn" {
      + address_space         = [
          + "10.0.0.0/16",
        ]
      + guid                  = (known after apply)
      + id                    = (known after apply)
      + location              = "westus"
      + name                  = "lsml-vitrual-network"
      + resource_group_name   = "lsml-resource-group"
      + subnet                = (known after apply)
      + vm_protection_enabled = false
    }

Plan: 2 to add, 0 to change, 0 to destroy.



  on common.tf line 4, in provider "a

Смотрим в интерфейс Azure в браузере - там должны были появиться наши ресурсы!

### Создание виртуальных машин в облаке

Создадим что-то более интересное в облаке.

In [19]:
! ls -la

total 24
drwxr-xr-x  3 root root 4096 Jan 13 12:28 .
drwxrwxr-x 12 1001 1002 4096 Jan 13 12:29 ..
-rw-r--r--  1 root root  723 Jan 13 12:26 common.tf
drwxr-xr-x  3 root root 4096 Jan 13 12:26 .terraform
-rwxrwxrwx  1 root root 1035 Jan 13 12:26 .terraform.lock.hcl
-rw-r--r--  1 root root 2229 Jan 13 12:28 terraform.tfstate


In [20]:
%%writefile common.tf

# Тут будут описаны общие ресурсы

provider "azurerm" {
  # Провайдер, с которым будем работать. В данном случае - Azure
  version = "=2.40.0"
  features {}
}

# Группа ресурсов. К ней будут линковаться все осталные ресурсы
resource "azurerm_resource_group" "lsml_rg" {
  name = "lsml-resource-group"
  location = "westus"
}

# Виртуальная сеть внутри облака
resource "azurerm_virtual_network" "lsml_vn" {
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location
    
  name = "lsml-vitrual-network"

  address_space = ["10.0.0.0/16"]
  # Пул адресов внутри сети
}

# Виртуальная подсеть внутри облака
resource "azurerm_subnet" "lsml_subnet" {
  resource_group_name = azurerm_resource_group.lsml_rg.name
  virtual_network_name = azurerm_virtual_network.lsml_vn.name
    
  name = "internal"
    
  address_prefixes = ["10.0.2.0/24"]
}

Overwriting common.tf


In [21]:
%%writefile vm.tf
# Здесь описываем ресурсы, необходимые для работы машины

# Публичный адрес машины
resource "azurerm_public_ip" "lsml_pub_ip" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-public-ip"

  allocation_method = "Dynamic" # Выдаем динамический ip
  idle_timeout_in_minutes = 30
  domain_name_label = "alexlsmlmachine" # Для удобства можно использовать DNS имя
}

# Сетевой интерфейс для нашей машины
resource "azurerm_network_interface" "lsml_ni" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-nic"

  ip_configuration {
    name = "internal"
    private_ip_address_allocation = "Dynamic"
    subnet_id = azurerm_subnet.lsml_subnet.id
    public_ip_address_id = azurerm_public_ip.lsml_pub_ip.id
  }
}

# Сама виртуальная машина
resource "azurerm_virtual_machine" "lsml_vm" {
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location

  name = "lsml-machine"

  vm_size = "Standard_F2" # 2 CPU 4 GB
  # Какие размеры вообще бывают - https://azure.microsoft.com/en-us/pricing/details/virtual-machines/series/

  network_interface_ids = [
    azurerm_network_interface.lsml_ni.id,
    # Подключаем к сети
  ]

  storage_image_reference {
    # Используем образ Ubuntu 16.04
    publisher = "Canonical"
    offer = "UbuntuServer"
    sku = "16.04-LTS"
    version = "latest"
  }

  os_profile {
    computer_name = "hostname"
    admin_username = "azureuser" # Пользователь и его пароль
    admin_password = "Password1234!"
  }
  os_profile_linux_config {
    disable_password_authentication = false
    ssh_keys {
      path = "/home/azureuser/.ssh/authorized_keys"
      key_data = file("~/id_rsa.pub") # Можем указать наш ключ, как ключ для авторизации на машине
    }
  }

  delete_os_disk_on_termination = true
  storage_os_disk {
    name = "main-disk"
    caching = "ReadWrite"
    create_option = "FromImage"
    managed_disk_type = "Standard_LRS"
    disk_size_gb = "200" # Указываем какой использовать основной жесткий диск и его размер
  }
}


# Созраним несколько важный значений в output. Их можно будет потом использовать для наших целей

data "azurerm_public_ip" "lsml_public_ip" {
  name = azurerm_public_ip.lsml_pub_ip.name
  resource_group_name = azurerm_virtual_machine.lsml_vm.resource_group_name
}

output "public_domain" {
  # Домен нашего сервера
  value = "${azurerm_public_ip.lsml_pub_ip.domain_name_label}.${azurerm_resource_group.lsml_rg.location}.cloudapp.azure.com"
}

output "public_ip" {
  # Публичный ip нашего сервера
  value = data.azurerm_public_ip.lsml_public_ip.ip_address
}

output "private_ip" {
  # Приватный ip нашего сервера
  value = azurerm_network_interface.lsml_ni.private_ip_address
}

Writing vm.tf


Terraform работает на уровне директории. Все файлы с расширением `tf` он воспринимает как одну большую конфигурацию.

Запускаем команду и развертываем все описанные ресурсы в облаке.

In [22]:
! echo "yes" | terraform apply

azurerm_resource_group.lsml_rg: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group]
azurerm_virtual_network.lsml_vn: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network]

An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create
 <= read (data resources)

Terraform will perform the following actions:

  # data.azurerm_public_ip.lsml_public_ip will be read during apply
  # (config refers to values not yet known)
 <= data "azurerm_public_ip" "lsml_public_ip"  {
      + allocation_method       = (known after apply)
      + domain_name_label       = (known after apply)
      + fqdn                    = (known after apply)
      + id                      = (known after apply)
      + idle_timeout_in_minutes = (known after apply)
      + ip_ad

azurerm_subnet.lsml_subnet: Creating...
azurerm_public_ip.lsml_pub_ip: Creating...
azurerm_subnet.lsml_subnet: Creation complete after 8s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network/subnets/internal]
azurerm_public_ip.lsml_pub_ip: Still creating... [10s elapsed]
azurerm_public_ip.lsml_pub_ip: Creation complete after 13s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/publicIPAddresses/lsml-public-ip]
azurerm_network_interface.lsml_ni: Creating...
azurerm_network_interface.lsml_ni: Creation complete after 10s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/networkInterfaces/lsml-nic]
azurerm_virtual_machine.lsml_vm: Creating...
azurerm_virtual_machine.lsml_vm: Still creating... [10s elapsed]
azurerm_virtual_machine.lsml_vm: Still creatin

Все должно было создаться - смотрим в консоли в браузере!

Параметры, которые мы специально сохранили, можно посмотреть с помощью `terraform output`

In [26]:
! terraform plan

azurerm_resource_group.lsml_rg: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group]
azurerm_virtual_network.lsml_vn: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network]
azurerm_public_ip.lsml_pub_ip: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/publicIPAddresses/lsml-public-ip]
azurerm_subnet.lsml_subnet: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network/subnets/internal]
azurerm_network_interface.lsml_ni: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/networkInterfaces/lsml-nic]
azurerm_virtual_ma

In [23]:
! terraform output

private_ip = "10.0.2.4"
public_domain = "alexlsmlmachine.westus.cloudapp.azure.com"
public_ip = "13.91.45.230"


In [24]:
! terraform output -raw public_ip

13.91.45.230

Можно попробовать подключиться к машине по ее публичному адресу, как мы это уже делали выше
```bash
ssh azureuser@40.118.191.81
```

Или

```bash
ssh azureuser@alexlsmlmachine.westus.cloudapp.azure.com
```

### Terraform provisioners

Машина готова к использованию, однако на ней все еще нет нужных библиотек. Чтобы руками не устанавливать все необходимое каждый раз, можно воспользоваться провизионерами.

После старта машины, terraform запустил все провизионеры по порядку. Каждый провизионер может запустить произвольные скрипты на машинах.

Добавим провизионеры для установки анаконды после создания машины.

In [17]:
# Провизионеры запускаются только один раз при создании ресурса. Для старых ресурсов они не работают
# Поэтому удалим все созданное. Это можно сделать одной командой terraform destroy

! echo "yes" | terraform destroy


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_network_interface.lsml_ni will be destroyed
  - resource "azurerm_network_interface" "lsml_ni" {
      - applied_dns_servers           = [] -> null
      - dns_servers                   = [] -> null
      - enable_accelerated_networking = false -> null
      - enable_ip_forwarding          = false -> null
      - id                            = "/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/networkInterfaces/lsml-nic" -> null
      - internal_domain_name_suffix   = "c3li4fbr0dqudeoah3eliwpbhc.dx.internal.cloudapp.net" -> null
      - location                      = "westus" -> null
      - name                          = "lsml-nic" -> null
      - private_ip_address            = "10.0.2.4" -> null
      - private_ip_addresses       

      - storage_account_name    = "lsmlstorage" -> null
    }

  # azurerm_subnet.lsml_subnet will be destroyed
  - resource "azurerm_subnet" "lsml_subnet" {
      - address_prefix                                 = "10.0.2.0/24" -> null
      - address_prefixes                               = [
          - "10.0.2.0/24",
        ] -> null
      - enforce_private_link_endpoint_network_policies = false -> null
      - enforce_private_link_service_network_policies  = false -> null
      - id                                             = "/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network/subnets/internal" -> null
      - name                                           = "internal" -> null
      - resource_group_name                            = "lsml-resource-group" -> null
      - virtual_network_name                           = "lsml-vitrual-network" -> null
    }

  # azurerm_virtual_mac

azurerm_storage_container.lsml_sc: Destroying... [id=https://lsmlstorage.blob.core.windows.net/content]
azurerm_role_assignment.example: Destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/providers/Microsoft.Authorization/roleAssignments/ccb6e779-4e81-4fc6-188a-87c18958c946]
azurerm_virtual_machine.lsml_vm: Destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Compute/virtualMachines/lsml-machine]
azurerm_role_assignment.example: Destruction complete after 3s
azurerm_storage_container.lsml_sc: Destruction complete after 3s
azurerm_storage_account.lsml_sa: Destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Storage/storageAccounts/lsmlstorage]
azurerm_virtual_machine.lsml_vm: Still destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-...t.Compute/virtualMachines/lsml-machine, 10s elapsed]
azurerm_storage_account.lsml_sa: D

In [18]:
%%writefile install-conda.sh

# get link from this site - https://www.anaconda.com/distribution/

wget https://repo.anaconda.com/archive/Anaconda3-2020.11-Linux-x86_64.sh -O install-conda.sh
chmod +x install-conda.sh
./install-conda.sh -b -p $HOME/anaconda3

# add conda to PATH

eval "$(/home/azureuser/anaconda3/bin/conda shell.bash hook)"
conda init

Overwriting install-conda.sh


In [22]:
%%writefile vm.tf
# Здесь описываем ресурсы, необходимые для работы машины

# Публичный адрес машины
resource "azurerm_public_ip" "lsml_pub_ip" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-public-ip"

  allocation_method = "Dynamic"
  # Выдаем динамический ip
  idle_timeout_in_minutes = 30
  domain_name_label = "alexlsmlmachine"
  # Для удобства можно использовать DNS имя
}

# Сетевой интерфейс для нашей машины
resource "azurerm_network_interface" "lsml_ni" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-nic"

  ip_configuration {
    name = "internal"
    private_ip_address_allocation = "Dynamic"
    subnet_id = azurerm_subnet.lsml_subnet.id
    public_ip_address_id = azurerm_public_ip.lsml_pub_ip.id
  }
}

# Сама виртуальная машина
resource "azurerm_virtual_machine" "lsml_vm" {
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location

  name = "lsml-machine"

  vm_size = "Standard_F2"
  # 2 CPU 4 GB
  # Какие размеры вообще бывают - https://azure.microsoft.com/en-us/pricing/details/virtual-machines/series/

  network_interface_ids = [
    azurerm_network_interface.lsml_ni.id,
    # Подключаем к сети
  ]

  storage_image_reference {
    # Используем образ Ubuntu 16.04
    publisher = "Canonical"
    offer = "UbuntuServer"
    sku = "16.04-LTS"
    version = "latest"
  }

  os_profile {
    computer_name = "hostname"
    admin_username = "azureuser"
    # Пользователь и его пароль
    admin_password = "Password1234!"
  }
  os_profile_linux_config {
    disable_password_authentication = false
    ssh_keys {
      path = "/home/azureuser/.ssh/authorized_keys"
      key_data = file("~/id_rsa.pub")
      # Можем указать наш ключ, как ключ для авторизации на машине
    }
  }

  delete_os_disk_on_termination = true
  storage_os_disk {
    name = "main-disk"
    caching = "ReadWrite"
    create_option = "FromImage"
    managed_disk_type = "Standard_LRS"
    disk_size_gb = "200"
    # Указываем какой использовать основной жесткий диск и его размер
  }

  provisioner "remote-exec" { # Запускаем удаленно
    script = "install-conda.sh" # Указываем, какой скрипт запустить

    connection {
      # Указываем, как подключиться к машине. Будем использовать ssh с паролем и доменным именем
      type = "ssh"
      user = "azureuser"
      password = "Password1234!"
      host = "${azurerm_public_ip.lsml_pub_ip.domain_name_label}.${azurerm_resource_group.lsml_rg.location}.cloudapp.azure.com"
    }
  }
}

# Созраним несколько важный значений в output. Их можно будет потом использовать для наших целей

data "azurerm_public_ip" "lsml_public_ip" {
  name = azurerm_public_ip.lsml_pub_ip.name
  resource_group_name = azurerm_virtual_machine.lsml_vm.resource_group_name
}

output "public_domain" {
  # Домен нашего сервера
  value = "${azurerm_public_ip.lsml_pub_ip.domain_name_label}.${azurerm_resource_group.lsml_rg.location}.cloudapp.azure.com"
}

output "public_ip" {
  # Публичный ip нашего сервера
  value = data.azurerm_public_ip.lsml_public_ip.ip_address
}

output "private_ip" {
  # Приватный ip нашего сервера
  value = azurerm_network_interface.lsml_ni.private_ip_address
}

Overwriting vm.tf


In [23]:
! echo "yes" | terraform apply


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create
 <= read (data resources)

Terraform will perform the following actions:

  # data.azurerm_public_ip.lsml_public_ip will be read during apply
  # (config refers to values not yet known)
 <= data "azurerm_public_ip" "lsml_public_ip"  {
      + allocation_method       = (known after apply)
      + domain_name_label       = (known after apply)
      + fqdn                    = (known after apply)
      + id                      = (known after apply)
      + idle_timeout_in_minutes = (known after apply)
      + ip_address              = (known after apply)
      + ip_version              = (known after apply)
      + location                = (known after apply)
      + name                    = "lsml-public-ip"
      + resource_group_name     = "lsml-resource-group"
      + reverse_fqdn            = (known after apply)
      + sku                     = (known aft

azurerm_resource_group.lsml_rg: Creating...
azurerm_resource_group.lsml_rg: Creation complete after 3s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group]
azurerm_virtual_network.lsml_vn: Creating...
azurerm_public_ip.lsml_pub_ip: Creating...
azurerm_virtual_network.lsml_vn: Still creating... [10s elapsed]
azurerm_public_ip.lsml_pub_ip: Still creating... [10s elapsed]
azurerm_public_ip.lsml_pub_ip: Creation complete after 12s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/publicIPAddresses/lsml-public-ip]
azurerm_virtual_network.lsml_vn: Creation complete after 12s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network]
azurerm_subnet.lsml_subnet: Creating...
azurerm_subnet.lsml_subnet: Creation complete after 7s [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resource

azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : python-3.8.5-h7579374_1.con
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : python-3.8.5-h7579374_1.con
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : prompt_toolkit-3.0.8-0.cond
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : chardet-3.0.4-py38_1003.con
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : openssl-1.1.1h-h7b6447c_0.c
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : dbus-1.13.18-hb2f20db_0.con
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : backports.functools_lru_cac
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : jdcal-1.4.1-py_0.conda:  10
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : json5-0.9.5-py_0.conda:  10
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : json5-0.9.5-py_0.conda:  11
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : conda-package-handling-1.7.
azurerm_virtual_machine.lsml_vm 

azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : brotlipy-0.7.0-py38h7b6447c
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : brotlipy-0.7.0-py38h7b6447c
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : harfbuzz-2.4.0-hca77d97_1.c
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : dask-2.30.0-py_0.conda:  32
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : pyyaml-5.3.1-py38h7b6447c_1
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : bleach-3.2.1-py_0.conda:  3
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : bleach-3.2.1-py_0.conda:  3
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : sqlite-3.33.0-h62c20be_0.co
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : numexpr-2.7.1-py38h423224d_
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : bokeh-2.2.3-py38_0.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : bokeh-2.2.3-py38_0.conda:
azurerm_virtual_machine.lsml_vm (rem

azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : itsdangerous-1.1.0-py_0.con
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : argh-0.26.2-py38_0.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : argh-0.26.2-py38_0.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : more-itertools-8.6.0-pyhd3e
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : backports-1.0-py_2.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : werkzeug-1.0.1-py_0.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : werkzeug-1.0.1-py_0.conda:
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : six-1.15.0-py38h06a4308_0.c
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : sphinxcontrib-1.0-py38_1.co
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : pyodbc-4.0.30-py38he6710b0_
azurerm_virtual_machine.lsml_vm (remote-exec): Extracting : pyodbc-4.0.30-py38he6710b0_
azurerm_virtual_machine.lsml_vm (remote-

...

azurerm_virtual_machine.lsml_vm (remote-exec):   backports.tempfile pkgs/main/noarch::backports.tem
azurerm_virtual_machine.lsml_vm (remote-exec): pfile-1.0-py_1
azurerm_virtual_machine.lsml_vm (remote-exec):   backports.weakref  pkgs/main/noarch::backports.weakref-1.0.post1-py_1
azurerm_virtual_machine.lsml_vm (remote-exec):   beautifulsoup4     pkgs/main/noarch::beautifulsoup4-4.9.3-pyhb0f4dca_0
azurerm_virtual_machine.lsml_vm (remote-exec):   bitarray           pkgs/main/linux-64::bitarray-1.6.1-py38h27cfd23_0
azurerm_virtual_machine.lsml_vm (remote-exec):   bkcharts           pkgs/main/linux-64::bkcharts-0.2-py38_0
azurerm_virtual_machine.lsml_vm (remote-exec):   blas               pkgs/main/linux-64::blas-1.0-mkl
azurerm_virtual_machine.lsml_vm (remote-exec):   bleach             pkgs/main/noarch::bleach-3.2.1-py_0
azurerm_virtual_machine.lsml_vm (remote-exec):   blosc              pkgs/main/linux-64::blosc-1.20.1-hd408876_0
azurerm_virtual_machine.lsml_vm (remote-exec):   bokeh  

azurerm_virtual_machine.lsml_vm (remote-exec):   ipython            pkgs/main/linux-64::ipython-7.19.0-py38hb070fc8_0
azurerm_virtual_machine.lsml_vm (remote-exec):   ipython_genutils   pkgs/main/linux-64::ipython_genutils-0.2.0-py38_0
azurerm_virtual_machine.lsml_vm (remote-exec):   ipywidgets         pkgs/main/noarch::ipywidgets-7.5.1-py_1
azurerm_virtual_machine.lsml_vm (remote-exec):   isort              pkgs/main/noarch::isort-5.6.4-py_0
azurerm_virtual_machine.lsml_vm (remote-exec):   itsdangerous       pkgs/main/noarch::itsdangerous-1.1.0-py_0
azurerm_virtual_machine.lsml_vm (remote-exec):   jbig               pkgs/main/linux-64::jbig-2.1-hdba287a_0
azurerm_virtual_machine.lsml_vm (remote-exec):   jdcal              pkgs/main/noarch::jdcal-1.4.1-py_0
azurerm_virtual_machine.lsml_vm (remote-exec):   jedi               pkgs/main/linux-64::jedi-0.17.1-py38_0
azurerm_virtual_machine.lsml_vm (remote-exec):   jeepney            pkgs/main/noarch::jeepney-0.5.0-pyhd3eb1b0_0
azurerm_virt

azurerm_virtual_machine.lsml_vm (remote-exec):   pyopenssl          pkgs/main/noarch::pyopenssl-19.1.0-py_1
azurerm_virtual_machine.lsml_vm (remote-exec):   pyparsing          pkgs/main/noarch::pyparsing-2.4.7-py_0
azurerm_virtual_machine.lsml_vm (remote-exec):   pyqt               pkgs/main/linux-64::pyqt-5.9.2-py38h05f1152_4
azurerm_virtual_machine.lsml_vm (remote-exec):   pyrsistent         pkgs/main/linux-64::pyrsistent-0.17.3-py38h7b6447c_0
azurerm_virtual_machine.lsml_vm (remote-exec):   pysocks            pkgs/main/linux-64::pysocks-1.7.1-py38_0
azurerm_virtual_machine.lsml_vm (remote-exec):   pytables           pkgs/main/linux-64::pytables-3.6.1-py38h9fd0a39_0
azurerm_virtual_machine.lsml_vm (remote-exec):   pytest             pkgs/main/linux-64::pytest-6.1.1-py38_0
azurerm_virtual_machine.lsml_vm (remote-exec):   python             pkgs/main/linux-64::python-3.8.5-h7579374_1
azurerm_virtual_machine.lsml_vm (remote-exec):   python-dateutil    pkgs/main/noarch::python-dateutil-2

azurerm_virtual_machine.lsml_vm (remote-exec): Preparing transaction: \
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)/
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)/
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)\
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)/
azurerm_virtual_machine.lsml_vm (remote-exec)\
azurerm_virtual_machine.lsml_vm (remote-exec)done
azurerm_virtual_machine.lsml_vm (remote-exec): Executing transaction: -
azurerm_virtual_machine.lsml_vm (remote-exec)\
azurerm

azurerm_virtual_machine.lsml_vm (remote-exec)/
azurerm_virtual_machine.lsml_vm (remote-exec)-
azurerm_virtual_machine.lsml_vm (remote-exec)\
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)/
azurerm_virtual_machine.lsml_vm (remote-exec)\
azurerm_virtual_machine.lsml_vm (remote-exec)|
azurerm_virtual_machine.lsml_vm (remote-exec)done
azurerm_virtual_machine.lsml_vm (remote-exec): installation finished.
azurerm_virtual_machine.lsml_vm (remote-exec): no change     /home/azureuser/anaconda3/condabin/conda
azurerm_virtual_machine.lsml_vm (remote-exec): no change     /home/azureuser/anaconda3/bin/conda
azurerm_virtual_machine.lsml_vm (remote-exec): no change     /home/azureuser/anaconda3/bin/conda-env
azurerm_virtual_machine.lsml_vm (remote-exec): no change     /home/azureuser/anaconda3/bin/activate
azurerm_virtual_machine.lsml_vm (remote-exec): no change     /home/azureuser/anaconda3/bin/deactivate
azurerm_virtual_machine.lsml_vm (remote-exec): no

In [24]:
! terraform output

private_ip = "10.0.2.4"
public_domain = "alexlsmlmachine.westus.cloudapp.azure.com"
public_ip = "104.42.71.2"


### Хранилище файлов

Когда мы говорим про машинное обучение, самое ценный компонент - это сами данные. Однако хранить их на жестком диске машине часто бывает слишком затратно - машина стоит относительно недешево и хочется избежать того, что она простаивает без дела, при этом не терять данные.

Один из подходов - это подключаемые жесткие диски, которые остаются в облаке даже после удаления машины. Однако для доступа к таким данных необходимо вновь арендовать новую машину и сам процесс управления такими дисками не слишком удобный. Поэтому мы пойдем другим путем, который окажется крайней полезными при дальшейшей работе с вычислительными кластерами.

В Azure есть специальный ресурс, который называется WASB - Windows Azure Storage Blob. Это специальное хранилище бинарных данных, в которое можно загрузить произвольное количество файлов произвольного размера. Таким образом, после работы на машине, результат можно выгрузить в это хранилище и к нему будет доступ из любого другого места.

Попробуем создать такое хранилище у нас в облаке.

In [29]:
%%writefile blob.tf
# Аккаунт хранения
resource "azurerm_storage_account" "lsml_sa" {
  name = "lsmlstorage"
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location
  account_tier = "Standard"
  account_replication_type = "LRS"
  allow_blob_public_access = true
  # По умолчанию доступ только приватный. Добавим публичный доступ.
}

# Конейнер внутри WASB
resource "azurerm_storage_container" "lsml_sc" {
  name = "content"
  # Как наш конейтнер с данными будет называться
  storage_account_name = azurerm_storage_account.lsml_sa.name
  container_access_type = "blob"
}


# По умолчанию у основного пользователя ограниченные права на работу с WASB.
# Выдалим доступ уровня "Storage Blob Data Contributor"
# data "azurerm_subscription" "primary" {
# }

# data "azurerm_client_config" "example" {
# }

# resource "azurerm_role_assignment" "example" {
#   scope = data.azurerm_subscription.primary.id
#   principal_id = data.azurerm_client_config.example.object_id

#   role_definition_name = "Storage Blob Data Contributor"
# }

Overwriting blob.tf


In [30]:
! echo "yes" | terraform apply

azurerm_resource_group.lsml_rg: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group]
azurerm_public_ip.lsml_pub_ip: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/publicIPAddresses/lsml-public-ip]
azurerm_virtual_network.lsml_vn: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network]
azurerm_storage_account.lsml_sa: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Storage/storageAccounts/lsmlstorage]
azurerm_subnet.lsml_subnet: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network/subnets/internal]
azurerm_network_int

Для работы с хранилищем можно использовать утилиту `az`, которую мы рассматривали выше.
Попробуем загрузить какой-то файл в наше облако. Так как мы выдали публичный доступ до нашего контейнера, то этот файл будет сразу же доступ всем в интернете.

In [31]:
! az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CGXUCU28F to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "21f26c24-0793-4b07-a73d-563cd2ec235f",
    "id": "7d1225ca-27cc-40b7-8036-c62a48072ba8",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Спонсорское предложение Microsoft Azure 2",
    "state": "Enabled",
    "tenantId": "21f26c24-0793-4b07-a73d-563cd2ec235f",
    "user": {
      "name": "adkosmachev@edu.hse.ru",
      "type": "user"
    }
  }
]


In [32]:
%%writefile file.txt
THIS IS MY FILE!

Writing file.txt


In [33]:
! cat file.txt

THIS IS MY FILE!


In [34]:
! az storage blob upload --account-name lsmlstorage --container-name content --file file.txt --name file-in-cloud.txt


There are no credentials provided in your command and environment, we will query for the account key inside your storage account. 
Please provide --connection-string, --account-key or --sas-token as credentials, or use `--auth-mode login` if you have required RBAC roles in your command. For more information about RBAC roles in storage, visit https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-rbac-cli. 
Setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information.
Finished[#############################################################]  100.0000%
{
  "etag": "\"0x8D8B7C1D68638A3\"",
  "lastModified": "2021-01-13T12:50:42+00:00"
}


In [35]:
! az storage blob list --account-name lsmlstorage --container-name content

There are no credentials provided in your command and environment, we will query for the account key inside your storage account. 
Please provide --connection-string, --account-key or --sas-token as credentials, or use `--auth-mode login` if you have required RBAC roles in your command. For more information about RBAC roles in storage, visit https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-rbac-cli. 
Setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information.
[
  {
    "container": "content",
    "content": "",
    "deleted": null,
    "encryptedMetadata": null,
    "encryptionKeySha256": null,
    "encryptionScope": null,
    "isAppendBlobSealed": null,
    "isCurrentVersion": null,
    "metadata": {},
    "name": "file-in-cloud.txt",
    "objectReplicationDestinationPolicy": null,
    "objectReplicationSourceProperties": [],
    "properties": {
      "appendBlobCommittedBlockCount": 

In [36]:
! wget https://lsmlstorage.blob.core.windows.net/content/file-in-cloud.txt

--2021-01-13 12:52:45--  https://lsmlstorage.blob.core.windows.net/content/file-in-cloud.txt
Resolving lsmlstorage.blob.core.windows.net (lsmlstorage.blob.core.windows.net)... 20.150.34.228
Connecting to lsmlstorage.blob.core.windows.net (lsmlstorage.blob.core.windows.net)|20.150.34.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17 [text/plain]
Saving to: ‘file-in-cloud.txt’

file-in-cloud.txt   100%[===================>]      17  --.-KB/s    in 0s      

2021-01-13 12:52:46 (5.08 MB/s) - ‘file-in-cloud.txt’ saved [17/17]



In [37]:
! cat file-in-cloud.txt

THIS IS MY FILE!


Таким образом мы умеет работать загружать файлы в облако и выгрузать их оттуда.

Вместо публичного доступа, можно оставить только приватный и получать файлы тоже через `az`

In [35]:
! az storage blob download --account-name lsmlstorage --container-name content --file file-from-az.txt --name file-in-cloud.txt

There are no credentials provided in your command and environment, we will query for the account key inside your storage account. 
Please provide --connection-string, --account-key or --sas-token as credentials, or use `--auth-mode login` if you have required RBAC roles in your command. For more information about RBAC roles in storage, visit https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-rbac-cli. 
Setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information.
Finished[#############################################################]  100.0000%
{
  "content": null,
  "deleted": false,
  "metadata": {},
  "name": "file-in-cloud.txt",
  "properties": {
    "appendBlobCommittedBlockCount": null,
    "blobTier": null,
    "blobTierChangeTime": null,
    "blobTierInferred": false,
    "blobType": "BlockBlob",
    "contentLength": 17,
    "contentRange": "bytes 0-16/17",
    "contentSettings": {

In [36]:
! cat file-from-az.txt

THIS IS MY FILE!


Если файлов значительно больше чем 1, то можно воспользоваться утилитой azcopy. Она умеет синхронизировать сразу целые директории между облаком и локальным диском.

In [38]:
! wget -O azcopy_v10.tar.gz https://aka.ms/downloadazcopy-v10-linux && tar -xf azcopy_v10.tar.gz --strip-components=1

--2021-01-13 12:54:17--  https://aka.ms/downloadazcopy-v10-linux
Resolving aka.ms (aka.ms)... 23.53.53.76
Connecting to aka.ms (aka.ms)|23.53.53.76|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://azcopyvnext.azureedge.net/release20201211/azcopy_linux_amd64_10.8.0.tar.gz [following]
--2021-01-13 12:54:17--  https://azcopyvnext.azureedge.net/release20201211/azcopy_linux_amd64_10.8.0.tar.gz
Resolving azcopyvnext.azureedge.net (azcopyvnext.azureedge.net)... 212.188.39.162, 212.188.39.208, 2a02:28:1:31::d4bc:278b, ...
Connecting to azcopyvnext.azureedge.net (azcopyvnext.azureedge.net)|212.188.39.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9367788 (8.9M) [application/gzip]
Saving to: ‘azcopy_v10.tar.gz’

azcopy_v10.tar.gz   100%[===================>]   8.93M  24.4MB/s    in 0.4s    

2021-01-13 12:54:18 (24.4 MB/s) - ‘azcopy_v10.tar.gz’ saved [9367788/9367788]



In [39]:
! mkdir -p big-folder
! echo file1 > big-folder/file1.txt
! echo file2 > big-folder/file2.txt
! echo file3 > big-folder/file3.txt

In [40]:
! ls big-folder/

file1.txt  file2.txt  file3.txt


Из-за особенностей устройства azcopy, команды ниже не работают внутри Docker. Если вам это ничего не говорит, то можете не беспокоиться и просто запускать команды. Если же вы запускаете их в ноутбуке из под Docker (как это люблю делать я), то просто делайте это вне Docker окружения.

In [41]:
! ./azcopy login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DDCVKFBQX to authenticate.

INFO: Logging in under the "Common" tenant. This will log the account in under its home tenant.
INFO: If you plan to use AzCopy with a B2B account (where the account's home tenant is separate from the tenant of the target storage account), please sign in under the target tenant with --tenant-id

Failed to perform login command: 
failed to get keyring during saving token, operation not permitted

NOTE: If your credential was created in the last 5 minutes, please wait a few minutes and try again.


In [42]:
! ./azcopy sync big-folder/ https://lsmlstorage.blob.core.windows.net/content/big-folder-in-cloud --recursive


Cannot perform sync due to error: Login Credentials missing. No SAS token or OAuth token is present and the resource is not public


In [46]:
! wget https://lsmlstorage.blob.core.windows.net/content/big-folder-in-cloud/file1.txt

--2021-01-06 12:40:39--  https://lsmlstorage.blob.core.windows.net/content/big-folder-in-cloud/file1.txt
Resolving lsmlstorage.blob.core.windows.net (lsmlstorage.blob.core.windows.net)... 52.239.162.4
Connecting to lsmlstorage.blob.core.windows.net (lsmlstorage.blob.core.windows.net)|52.239.162.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6 [text/plain]
Saving to: ‘file1.txt’

file1.txt           100%[===================>]       6  --.-KB/s    in 0s      

2021-01-06 12:40:40 (1.37 MB/s) - ‘file1.txt’ saved [6/6]



In [47]:
! mkdir new-folder

In [ ]:
! ./azcopy sync https://lsmlstorage.blob.core.windows.net/content/big-folder-in-cloud new-folder/ --recursive

### Выключайте ресурсы!

Использование облака - дело не самое дешевое. Поэтому после работы в облаке, сохраните свой результат и отключите вычислительные ресурсы.

<img src="https://avatars.mds.yandex.net/get-zen_doc/27036/pub_5b7714d46cf6c700aa7e06f7_5b7714d99dce6b00aa88d3f6/scale_1200">

Это можно сделать как минимум тремя способами:
1. Выключить машины руками через браузер
2. Удалить\закоментировать ненужные ресурсы в конфигурации terraform и применить `terraform apply`
3. Точечно удалять ресурсы через `terraform destroy -target <target>`. Например, `terraform destroy -target azurerm_virtual_machine.lsml_vm` точечно убьем машину в облаке. После повторного применения `terraform apply` она вновь появится.

**Не запускайте `terraform destroy` не подумав!** Эта команда уничтожит все ресурсы в облаке, **включая хранилище данных, где могут быть ценные для вас файлы**.

Следите за уровнем доступных вам кредитов и за количеством арендованных ресурсов, чтобы их хватило на все работы в рамках курса!

In [43]:
! cat vm.tf

# Здесь описываем ресурсы, необходимые для работы машины

# Публичный адрес машины
resource "azurerm_public_ip" "lsml_pub_ip" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-public-ip"

  allocation_method = "Dynamic" # Выдаем динамический ip
  idle_timeout_in_minutes = 30
  domain_name_label = "alexlsmlmachine" # Для удобства можно использовать DNS имя
}

# Сетевой интерфейс для нашей машины
resource "azurerm_network_interface" "lsml_ni" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-nic"

  ip_configuration {
    name = "internal"
    private_ip_address_allocation = "Dynamic"
    subnet_id = azurerm_subnet.lsml_subnet.id
    public_ip_address_id = azurerm_public_ip.lsml_pub_ip.id
  }
}

# Сама виртуальная машина
resource "azurerm_virtual_machine" "lsml_vm" {
  resource_group_name = azurerm_resource_group.lsml_rg.name
  

In [46]:
%%writefile vm.tf
# Здесь описываем ресурсы, необходимые для работы машины

# Публичный адрес машины
resource "azurerm_public_ip" "lsml_pub_ip" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-public-ip"

  allocation_method = "Dynamic" # Выдаем динамический ip
  idle_timeout_in_minutes = 30
  domain_name_label = "alexlsmlmachine" # Для удобства можно использовать DNS имя
}

# Сетевой интерфейс для нашей машины
resource "azurerm_network_interface" "lsml_ni" {
  location = azurerm_resource_group.lsml_rg.location
  resource_group_name = azurerm_resource_group.lsml_rg.name

  name = "lsml-nic"

  ip_configuration {
    name = "internal"
    private_ip_address_allocation = "Dynamic"
    subnet_id = azurerm_subnet.lsml_subnet.id
    public_ip_address_id = azurerm_public_ip.lsml_pub_ip.id
  }
}

# # Сама виртуальная машина
# resource "azurerm_virtual_machine" "lsml_vm" {
#   resource_group_name = azurerm_resource_group.lsml_rg.name
#   location = azurerm_resource_group.lsml_rg.location

#   name = "lsml-machine"

#   vm_size = "Standard_F2" # 2 CPU 4 GB
#   # Какие размеры вообще бывают - https://azure.microsoft.com/en-us/pricing/details/virtual-machines/series/

#   network_interface_ids = [
#     azurerm_network_interface.lsml_ni.id,
#     # Подключаем к сети
#   ]

#   storage_image_reference {
#     # Используем образ Ubuntu 16.04
#     publisher = "Canonical"
#     offer = "UbuntuServer"
#     sku = "16.04-LTS"
#     version = "latest"
#   }

#   os_profile {
#     computer_name = "hostname"
#     admin_username = "azureuser" # Пользователь и его пароль
#     admin_password = "Password1234!"
#   }
#   os_profile_linux_config {
#     disable_password_authentication = false
#     ssh_keys {
#       path = "/home/azureuser/.ssh/authorized_keys"
#       key_data = file("~/id_rsa.pub") # Можем указать наш ключ, как ключ для авторизации на машине
#     }
#   }

#   delete_os_disk_on_termination = true
#   storage_os_disk {
#     name = "main-disk"
#     caching = "ReadWrite"
#     create_option = "FromImage"
#     managed_disk_type = "Standard_LRS"
#     disk_size_gb = "200" # Указываем какой использовать основной жесткий диск и его размер
#   }
# }


# # Созраним несколько важный значений в output. Их можно будет потом использовать для наших целей

# data "azurerm_public_ip" "lsml_public_ip" {
#   name = azurerm_public_ip.lsml_pub_ip.name
#   resource_group_name = azurerm_virtual_machine.lsml_vm.resource_group_name
# }

# output "public_domain" {
#   # Домен нашего сервера
#   value = "${azurerm_public_ip.lsml_pub_ip.domain_name_label}.${azurerm_resource_group.lsml_rg.location}.cloudapp.azure.com"
# }

# output "public_ip" {
#   # Публичный ip нашего сервера
#   value = data.azurerm_public_ip.lsml_public_ip.ip_address
# }

# output "private_ip" {
#   # Приватный ip нашего сервера
#   value = azurerm_network_interface.lsml_ni.private_ip_address
# }

Overwriting vm.tf


In [49]:
! echo "yes" | terraform destroy


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_network_interface.lsml_ni will be destroyed
  - resource "azurerm_network_interface" "lsml_ni" {
      - applied_dns_servers           = [] -> null
      - dns_servers                   = [] -> null
      - enable_accelerated_networking = false -> null
      - enable_ip_forwarding          = false -> null
      - id                            = "/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/networkInterfaces/lsml-nic" -> null
      - internal_domain_name_suffix   = "2kfcj2iu45dephzegvk5ose2uc.dx.internal.cloudapp.net" -> null
      - location                      = "westus" -> null
      - mac_address                   = "00-0D-3A-36-93-5F" -> null
      - name                          = "lsml-nic" -> null
      - private_ip_address

azurerm_storage_container.lsml_sc: Destroying... [id=https://lsmlstorage.blob.core.windows.net/content]
azurerm_virtual_machine.lsml_vm: Destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Compute/virtualMachines/lsml-machine]
azurerm_storage_container.lsml_sc: Destruction complete after 3s
azurerm_storage_account.lsml_sa: Destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Storage/storageAccounts/lsmlstorage]
azurerm_virtual_machine.lsml_vm: Still destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-...t.Compute/virtualMachines/lsml-machine, 10s elapsed]
azurerm_storage_account.lsml_sa: Destruction complete after 9s
azurerm_virtual_machine.lsml_vm: Still destroying... [id=/subscriptions/7d1225ca-27cc-40b7-8036-...t.Compute/virtualMachines/lsml-machine, 20s elapsed]
azurerm_virtual_machine.lsml_vm: Still destroying... [id=/subscriptions/7